In [1]:
from utils import *
import os.path
from scipy.stats import qmc
from tqdm import tqdm
from pymoo.operators.sampling.lhs import LHS
import polars as pl

In [2]:
save_dir = 'scale_problem'
create_directory_if_not_exist(save_dir)

In [3]:
problem_dim = 5
n_runs = 30
n_evals = 1000
seed=42
n_instance_reapets = 3

In [4]:
df = pl.read_parquet(f'{save_dir}/*.parquet')
df

problem,instance,y,scale,x_0,x_1,x_2,x_3,x_4
i64,i64,f64,f64,f64,f64,f64,f64,f64
1,1,6.060719,0.041858,-4.296427,-3.104281,-0.192616,-0.673033,3.131601
1,1,6.126361,0.041858,4.528078,-3.067798,2.361273,0.416028,3.076782
1,1,3.877255,0.041858,-1.140813,-3.502411,-2.387861,1.560439,-1.006926
1,1,6.632123,0.041858,2.961459,2.547677,-0.053763,2.266459,4.891365
1,1,4.10911,0.041858,-2.809439,-0.757153,-0.830928,0.079946,-0.28717
…,…,…,…,…,…,…,…,…
24,15,7190.788579,29.199993,4.132589,1.248377,-1.640549,3.33526,-1.551975
24,15,8519.918924,29.199993,4.488035,2.981614,-3.87305,3.56724,-0.802508
24,15,8824.975638,29.199993,-3.955833,-1.963122,2.818195,-2.607644,-3.413899


In [5]:
ela_save_dir = f'{save_dir}/dummy'
create_directory_if_not_exist(ela_save_dir)

In [ ]:
for r in tqdm(df.select(['problem', 'instance', 'scale']).unique().sort(['instance']).rows(named=True)):
    instance = r['instance']
    problem = r['problem']
    scale = r['scale']
    save_file = f'{ela_save_dir}/p_{problem}__i_{instance}__s_{scale}.parquet'
    
    #if os.path.exists(save_file):
    #    continue
        
    sdf = df.lazy().filter(pl.col('problem')==problem).filter(pl.col('instance')==instance).filter(pl.col('scale')==scale).collect()
    y = sdf['y'].to_numpy()
    
    features = {}
    features['y_max_minus_min'] = np.abs(y.max()-y.min())
    # print(y.shape, features)
    
    features['problem'] = problem
    features['instance'] = instance
    features['scale'] = scale
    
    features = pl.DataFrame([features])
    features = features.with_columns([
        pl.col('*').cast(pl.Float64)
    ]).with_columns([
        pl.col('problem').cast(pl.Int64),  # Keep 'problem' as Int64
        pl.col('instance').cast(pl.Int64)  # Keep 'instance' as Int64
    ])
    
    features.write_parquet(save_file)

 47%|████▋     | 50254/108000 [4:53:12<6:01:38,  2.66it/s] 

In [ ]:
dfg=dfg

In [ ]:
#suite = list(generate_scaled_problems(problem_dim=problem_dim, seed=seed, n_runs=n_instance_reapets))

In [ ]:
#sdfsdf=sdf

In [ ]:
#runs_dir = f'{save_dir}/runs'
#create_directory_if_not_exist(runs_dir)

In [ ]:
for problem in tqdm(suite):
    file_path = f'{runs_dir}/p_{problem.id_function}__i_{problem.id_instance}__d_{problem_dim}__fe_{n_evals}__scale_{problem.scale}.parquet'
    
    if os.path.exists(file_path):
        continue
    
    #optimum = problem.final_target_fvalue1*scalar
    #print(optimum)
    #pw = COCOProblemWrapperScale(problem, scalar)
    
    opt = problem.evaluate(problem.pareto_set())[0, 0]
    pdf = pl.DataFrame(run_algorithms(problem, n_runs=n_runs, n_eval=n_evals)).with_columns([
        pl.lit(problem.id_function).alias("problem"),
        pl.lit(problem.id_instance).alias("instance"),
        pl.lit(problem.scale).alias("scale"),
        pl.lit(opt).alias("optimum"),
    ])
    pdf.write_parquet(file_path)
    
    #print(pdf)
    #print(problem.best_observed_fvalue1*scalar)
    #break

In [ ]:
#with pl.Config(set_fmt_float="full"):
#    v = pdf.with_columns([
#        (pl.col('GA')-pl.col('optimum')).alias('GA_precision'),
#        (pl.col('DE')-pl.col('optimum')).alias('DE_precision'),
#        (pl.col('PSO')-pl.col('optimum')).alias('PSO_precision'),
#        (pl.col('CMAES')-pl.col('optimum')).alias('CMAES_precision'),
#        (pl.col('ES')-pl.col('optimum')).alias('ES_precision')
#    ]).select(['GA_precision', 'DE_precision', 'PSO_precision', 'CMAES_precision', 'ES_precision'])
#    print(v)

In [ ]:
problem.final_target_fvalue1()

In [ ]:
#dir(problem)

In [ ]:
df = pl.read_parquet(f'{runs_dir}/*').sort(["problem", "instance"])
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(data=df.filter(pl.col('problem').is_in([2,24])), x='PSO', y='scalar', hue='problem')

plt.xscale('log')
plt.yscale('log')

In [ ]:
df_samples = pl.read_parquet(f'{save_dir}/*.parquet').sort(["problem", "instance"])
df_samples